# Fitting a FLMP
In this notebook, we will fit a Fuzzy Logical Model of Perception (FLMP) to audiovisual speech perception data.

In [2]:
import numpy as np
from os import path
from glob import glob

import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

from scipy.stats import binom
from scipy.optimize import minimize, LinearConstraint, Bounds

## Prepare Data
The data consists of five text files, each containing seven rows and five columns, where:
- Row 1: Audiotorial data
- Row 2: Visual data
- Rows 3-7: Audiovisual data
    - a combination of rows 1 and 2
    - visual goes from 'b' (row 3) to 'd' (row 7) 
    - audio goes from 'b' (col 1) to 'd' (col 5)

In [3]:
# get paths to data files
file_paths = glob("./data/*.txt")

# load all data into a single array
data = np.array([np.loadtxt(fname) for fname in file_paths])
N, M, K = data.shape

# define number of samples for each subject
n_samples = 24 

# split into the three different data types
# data_A = data[:, 0, :]
# data_V = data[:, 1, :]
# data_AV = data[:, 2:, :]

## Fit FLMP

In [4]:
# create softmax function
def softmax(x):
    x = np.concatenate([
        np.array([x]).flatten(), [0]
    ])
    e = np.exp(x)
    return e[0] / e.sum()

In [5]:
def objective_function(theta, data, K_, n_samples=24):
    # extract audio and visual parameters
    theta_A = theta[0:K_]
    theta_V = theta[K_: ]

    # get probabilities for audio and visual
    p_A = np.array([softmax(t) for t in theta_A]).reshape(-1,1)
    p_V = np.array([softmax(t) for t in theta_V]).reshape(-1,1)
    
    # compute audiovisual probabilities by
    # taking the outer product for all combinations of audio and visual
    p_AV = (p_A @ p_V.T) / (p_A @ p_V.T + (1 - p_A) @ (1 - p_V).T)

    # compute the log-likelihoods
    L_A = binom.logpmf(data[0], n_samples, p_A).sum()
    L_V = binom.logpmf(data[1], n_samples, p_V).sum()
    L_AV = binom.logpmf(data[2:], n_samples, p_AV).sum()
    L = L_A + L_V + L_AV

    # return the negative log-likelihood
    return -L

In [6]:
def flmp_fit(data, n_samples, K_):
    """Perform FLMP fit to data (7, 5) for a single subject"""
    theta = np.random.randn(K_*2)
    opt_result = minimize(objective_function, theta, args=(data, K_, n_samples))
    objective, theta_A, theta_V = (
        opt_result.fun, 
        (opt_result.x[0:K_]), 
        (opt_result.x[K_:])
    )
    return objective, theta_A, theta_V

In [7]:
# perform FLMP fit for each subject
theta_A = np.zeros((K, N))
theta_V = np.zeros((K, N))
neg_L = np.zeros(K)
for i in range(K):
    obj, tA, tV = flmp_fit(data[i], n_samples, K)

    # print results
    print(f'\nResults for subject {i+1}')
    print(f'Negative log-likelihood: {obj}')
    print("Audio:", tA)
    print("Visual:", tV)
    neg_L[i] = obj
    theta_A[i, :] = tA
    theta_V[i, :] = tV


Results for subject 1
Negative log-likelihood: 759.1766497785582
Audio: [-1.30064678 -0.58587863 -0.03341144  0.15842417  0.21756657]
Visual: [0.31729893 0.63992323 0.86817324 1.56726433 1.73585199]

Results for subject 2
Negative log-likelihood: 745.9722540530345
Audio: [-1.39272991 -0.29460938  0.25587786  0.59689644  0.66610568]
Visual: [0.39729955 0.82350244 1.08317773 1.32397932 1.59814067]

Results for subject 3
Negative log-likelihood: 657.7838496588924
Audio: [-0.6320468  -0.30334676  0.06146448  0.26914399  0.36668503]
Visual: [-0.06229033  0.15831725  0.59669425  1.40463184  1.71854358]

Results for subject 4
Negative log-likelihood: 742.0495449184498
Audio: [-0.65235526 -0.49645722 -0.3829699  -0.21665941  0.03682485]
Visual: [-0.89757262 -0.74897395 -0.51324377  0.72452205  1.13194631]

Results for subject 5
Negative log-likelihood: 805.7723566088322
Audio: [-0.84433938 -0.49486807 -0.36327183 -0.19551536 -0.08375172]
Visual: [-0.32344658 -0.14628076  0.5562403   1.5803675